<a href="https://colab.research.google.com/github/0xrocky/master-thesis/blob/main/neural-model-training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Generic imports
import numpy as np
from numpy.core.fromnumeric import shape
from google.colab import drive
from tqdm import tqdm
import random
import matplotlib.pyplot as plt

# Neural network imports
import tensorflow as tf
from tensorflow.keras.utils import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import *

In [ ]:
# Load traces from the GDrive folder
LOCAL_DRIVE = '/content/drive'
drive.mount( LOCAL_DRIVE, force_remount=True )
CYCLES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B5-2021-11-22-16.44-cycles.npy'
TRACES_FILE = '/content/drive/MyDrive/ECCML/RealTraces/B5-2021-11-22-16.44-traces.npy'

traces = np.load( TRACES_FILE, allow_pickle=True )
cycles = np.load( CYCLES_FILE, allow_pickle=True )
traces = np.multiply( traces, 175 )

Mounted at /content/drive


In [ ]:
# Macro
REAL_TRACES_NUMBER = 1 # number of traces
CONT_TRUE_SAMPLES = 546751  # number of SUB samples that I will pick up and I will tag TRUE, until 10 mln
ID_NUMBER = 3437  # number of multiplication in a single trace
ID_TIME = 153  # time duration of a single add

In [ ]:
# Per mia comodità costruisco delle funzioni per ottenere un sample, un tag, i cicli di clock
def get_trace_sample( i ):
  return traces[ 0 ][ i ]
  #return 0

def get_cycles_tag( i ):
  return cycles[ 0 ][ i ][ 0 ]

def get_cycles_clock( i ):
  return cycles[ 0 ][ i ][ 1 ]

In [ ]:
# NOT SHUFFLE
# Per tutti i tag nell'array, filtro solo sui tag dell'ADD e della MUL
trace_samples_tags = []
cont_false = 0 # contatore per le MUL: devo fermarmi prima di prenderle tutte per equilibrare il dataset

# Costruisco il vettore di rilevazioni di ID e MUL, appendendo ad ogni sample di ciascuna operazione un tag booleano
def appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, bool_tag ):
  for j in range( operation_time ):
    trace_samples_tags += [ ( get_trace_sample( clock_cicles_op_curr + j ), bool_tag ) ]

for h in range( REAL_TRACES_NUMBER ):
  for i in range( len( cycles[ 0 ] ) - 1 ):
    tag_op_curr = get_cycles_tag( i )       # il tag dell'operazione corrente
    clock_cicles_op_curr = get_cycles_clock( i )   # i cicli di clock da cui parte l'operazione corrente
    operation_time = get_cycles_clock( i + 1 ) - clock_cicles_op_curr # calcolo la durata di un'operazione come la differenza tra i cicli di clock registrati all'inizio dell'operazione successiva e quelli dell'operazione corrente - 1
    if clock_cicles_op_curr + operation_time >= len( traces[ 0 ] ):  # il numero di samples è 10mln, ma il numero di cicli di clock registrati è maggiore
      print( i, "break" )
      break
    if tag_op_curr == 4:                  # sottrazione
      appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, True )
    else:                                 # tutto il resto
      if cont_false > CONT_TRUE_SAMPLES: # il dataset deve essere equilibrato, quindi prendo un numero di False samples proporzionale a quello delle SUB
        continue                          # se ho già prelevato samples a sufficienza delle altre operazioni, ignoro
      else:
        cont_false += operation_time       # tengo traccia dei False samples che ora aggiungerò al dataset
        appending_bool_flag_at_samples( trace_samples_tags, operation_time, clock_cicles_op_curr, False )

6337 break


In [ ]:
print( "Quanti False samples ho preso:", cont_false )
print( "Dimensione del vettore delle tracce:", shape( trace_samples_tags ) )
print( trace_samples_tags[ :100 ] )
cycles = []
traces = []

Quanti False samples ho preso: 548051
Dimensione del vettore delle tracce: (1094802, 2)
[(-0.8544921875, False), (0.341796875, False), (0.8544921875, False), (3.5888671875, False), (0.68359375, False), (-0.1708984375, False), (1.025390625, False), (-4.1015625, False), (3.076171875, False), (3.076171875, False), (0.68359375, False), (1.1962890625, False), (0.1708984375, False), (-1.8798828125, False), (2.5634765625, False), (0.8544921875, False), (-0.1708984375, False), (0.341796875, False), (-2.2216796875, False), (2.5634765625, False), (1.3671875, False), (3.2470703125, False), (1.3671875, False), (-1.3671875, False), (0.68359375, False), (3.759765625, False), (0.0, False), (-0.68359375, False), (2.5634765625, False), (1.1962890625, False), (2.05078125, False), (1.3671875, False), (2.5634765625, False), (0.1708984375, False), (-1.3671875, False), (1.708984375, False), (1.3671875, False), (-1.1962890625, False), (-4.1015625, False), (5.810546875, False), (4.78515625, False), (-1.025390

In [ ]:
temp = 0
for i in range( len( trace_samples_tags ) ):
  if trace_samples_tags[ i ][ 1 ] == True:
    temp += 1
print( "Numero di True samples:", temp )
temp = 0
for i in range( len( trace_samples_tags ) ):
  if trace_samples_tags[ i ][ 1 ] == False:
    temp += 1
print( "Numero di False samples:", temp )

Numero di True samples: 546751
Numero di False samples: 548051


In [ ]:
trace_samples = []
trace_tags = []

for single_trace in range( REAL_TRACES_NUMBER ):
  temp_trace_sample = []
  temp_trace_tag = []
  for smpl_tag in range( len( trace_samples_tags ) ):
    temp_trace_sample += [ trace_samples_tags[ smpl_tag ][ 0 ] ]
    temp_trace_tag += [ trace_samples_tags[ smpl_tag ][ 1 ] ]
  trace_samples += [ temp_trace_sample ]
  trace_tags += [ temp_trace_tag ]

size_trace_samples = len( trace_samples )
size_trace_tags = len( trace_tags )
size_single_trace_samples = len( trace_samples[ 0 ] )
size_single_trace_tags = len( trace_tags[ 0 ] )

print( "Size of the list of all real traces (numbers of executions):", REAL_TRACES_NUMBER )
print( "Size of the list of all traces samples constructed from the real traces:", size_trace_samples )
print( "Size of the list of all traces tags constructed from the real traces:", size_trace_tags )
print( "Size of the only first samples trace", size_single_trace_samples )
print( "Size of the only first tags trace", size_single_trace_tags )

if REAL_TRACES_NUMBER != size_trace_samples or REAL_TRACES_NUMBER != size_trace_tags:
  print( "ERROR: number of total samples not equal at number of total tags in the real traces" )
if size_single_trace_samples != size_single_trace_tags:
  print( "ERROR: number of samples of the first trace not equal at number of tags of the first trace" )

Size of the list of all real traces (numbers of executions): 1
Size of the list of all traces samples constructed from the real traces: 1
Size of the list of all traces tags constructed from the real traces: 1
Size of the only first samples trace 1094802
Size of the only first tags trace 1094802


In [ ]:
# Sliding window to create the dataset: fixing a window size and an advancement step, the aim is focusing on an Hamming weight window and tag
# with a 1 the window contains a TRUE tag, 0 otherwise
window = 500
step = 10
X_full_dataset = []
Y_full_dataset = []

for trace in range( REAL_TRACES_NUMBER ):
  for i in range( 0, size_single_trace_samples - window + 1, step ):
    X_full_dataset += [ trace_samples[ trace ][ i:i + window ] ]
    window_of_tags = np.array( trace_tags[ trace ][ i:i + window ] )
    set_True = 0
    # It detects where chunks begin, has some logic for the first and last chunk, and simply computes differences between chunk starts and discards lengths corresponding to False chunks
    arr = np.diff( np.where( np.concatenate( ( [ window_of_tags[ 0 ] ], window_of_tags[ :-1 ] != window_of_tags[ 1: ], [ True ] ) ) )[ 0 ] )[ ::2 ]
    if np.size( arr ) > 0 and np.max( arr ) >= ID_TIME:
      set_True = 1
    Y_full_dataset += [ set_True ]

size_X_full_dataset = len( X_full_dataset )
size_Y_full_dataset = len( Y_full_dataset )
size_single_trace_X = len( X_full_dataset[ 0 ] )
print( X_full_dataset[ :10 ] )
print( Y_full_dataset[ :100 ] )
print( "Size of the X dataset of {} sliding window of samples:".format( window ), size_X_full_dataset )
print( "Size of the Y dataset of {} sliding window of tags:".format( window ), size_Y_full_dataset )
print( "Size of the first trace of the X dataset of samples in sliding window:", size_single_trace_X )

[[-0.8544921875, 0.341796875, 0.8544921875, 3.5888671875, 0.68359375, -0.1708984375, 1.025390625, -4.1015625, 3.076171875, 3.076171875, 0.68359375, 1.1962890625, 0.1708984375, -1.8798828125, 2.5634765625, 0.8544921875, -0.1708984375, 0.341796875, -2.2216796875, 2.5634765625, 1.3671875, 3.2470703125, 1.3671875, -1.3671875, 0.68359375, 3.759765625, 0.0, -0.68359375, 2.5634765625, 1.1962890625, 2.05078125, 1.3671875, 2.5634765625, 0.1708984375, -1.3671875, 1.708984375, 1.3671875, -1.1962890625, -4.1015625, 5.810546875, 4.78515625, -1.025390625, 0.8544921875, 0.341796875, -0.341796875, -2.05078125, 3.759765625, 2.392578125, 0.341796875, 1.5380859375, 0.5126953125, -1.025390625, 5.9814453125, 2.2216796875, -0.1708984375, 1.8798828125, 1.5380859375, 0.68359375, 2.5634765625, 3.2470703125, 1.025390625, 0.8544921875, -0.341796875, -4.443359375, 6.3232421875, 2.2216796875, -0.341796875, 2.2216796875, -0.1708984375, -0.8544921875, 2.9052734375, -3.9306640625, 3.41796875, 2.9052734375, -0.3417968

In [ ]:
yes = no = 0
for i in range( size_Y_full_dataset ):
  if Y_full_dataset[ i ] == 1:
    yes += 1
  else:
    no += 1
print( "finestre taggate a 1", yes )
print( "finestre taggate a 0", no )

finestre taggate a 1 57626
finestre taggate a 0 51805


In [ ]:
X_full_dataset = np.array( X_full_dataset )
X_full_dataset = np.reshape( X_full_dataset, ( X_full_dataset.shape[ 0 ], X_full_dataset.shape[ 1 ], 1 ) )
Y_full_dataset = to_categorical( np.array( Y_full_dataset ) )
print( "Shapes:" )
print( "\tX_full_dataset" + str( X_full_dataset.shape ) )
print( "\tY_full_dataset" + str( Y_full_dataset.shape ) )

Shapes:
	X_full_dataset(109431, 500, 1)
	Y_full_dataset(109431, 2)


In [ ]:
# Building the LSTM
model = Sequential()
model.add( Conv1D( filters=64, kernel_size=3, activation='relu', input_shape=( X_full_dataset.shape[ 1 ], X_full_dataset.shape[ 2 ] ) ) )
model.add( AveragePooling1D( pool_size=10 ) )
model.add( LSTM( 1000 ) )
model.add( Dropout( 0.5 ) )
model.add( Dense( 1000, activation='relu' ) )
model.add( Dense( 2, activation='softmax' ) )
model.compile( loss='categorical_crossentropy', optimizer='SGD', metrics=['accuracy'] )
model.build()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 498, 64)           256       
                                                                 
 average_pooling1d (AverageP  (None, 49, 64)           0         
 ooling1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 1000)              4260000   
                                                                 
 dropout (Dropout)           (None, 1000)              0         
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 2)                 2002      
                                                        

In [ ]:
model.fit( X_full_dataset, Y_full_dataset, epochs=50, batch_size=64, validation_split=0.20 )

Epoch 1/50
1368/1368 [==============================] - 123s 88ms/step - loss: 0.5446 - accuracy: 0.7007 - val_loss: 0.1201 - val_accuracy: 0.9755
Epoch 2/50
1368/1368 [==============================] - 120s 88ms/step - loss: 0.1965 - accuracy: 0.9283 - val_loss: 0.0260 - val_accuracy: 0.9971
Epoch 3/50
1368/1368 [==============================] - 119s 87ms/step - loss: 0.1615 - accuracy: 0.9435 - val_loss: 0.0132 - val_accuracy: 0.9998
Epoch 4/50
1368/1368 [==============================] - 119s 87ms/step - loss: 0.1526 - accuracy: 0.9469 - val_loss: 0.0745 - val_accuracy: 0.9818
Epoch 5/50
1368/1368 [==============================] - 120s 88ms/step - loss: 0.1469 - accuracy: 0.9496 - val_loss: 0.0226 - val_accuracy: 0.9970
Epoch 6/50
1368/1368 [==============================] - 122s 89ms/step - loss: 0.1415 - accuracy: 0.9516 - val_loss: 0.0267 - val_accuracy: 0.9979
Epoch 7/50
1368/1368 [==============================] - 121s 89ms/step - loss: 0.1390 - accuracy: 0.9527 - val_loss: 0

In [ ]:
# Save model
print("Saving model...")
model.save("/content/drive/My Drive/ECCML/RealTraces/Models/real_model_LSTM1000_AVGPOOL_allOP_notshuffle.model")
print("Done.")

Saving model...


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECCML/RealTraces/Models/real_model_LSTM1000_AVGPOOL_allOP_notshuffle.model/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECCML/RealTraces/Models/real_model_LSTM1000_AVGPOOL_allOP_notshuffle.model/assets


Done.
